In [59]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/')
import numpy as np
import pandas as pd
import re
from collections import defaultdict
import csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
def preprocess(file): # read skills and make them into a dictionary with links as keys
  i = 0
  fails = ["sorry","unable","apologize","cannot", "frameworks languages softwares","does not contain","no technical skills","do not have access","does not mention anything"]
  with open(file) as f:
    reader = csv.reader(f)
    header = next(reader, None)  # skip table head
    all_skills = set()
    for skills in reader:
      #if i > 10000: break
      if i%100000==0:
        print(i,"rows processed")
      i+=1
      skills = [x.strip().replace("\"","").lower() for x in skills]
      link = skills[0]
      skills = skills[1].split(",")
      skills = set([s for s in skills if not any(f in s for f in fails)])
      skills_dict[link] = skills
      for skill in skills:
        all_skills.add(skill)
  return skills_dict, all_skills

In [129]:
def get_jobs_info(file, skills_dict,user_skills):
  input = set([x.strip().lower() for x in user_skills])
  fitting_jobs = set()
  skills_missing_1 = dict()
  skills_missing_2 = dict()
  skills_missing_3 = dict()
  input_len = len(input)
  for key in skills_dict:
    diff = set(skills_dict[key]) - input
    diff = set([x.strip() for x in diff])
    if len(diff) <= 3: # the difference in required skills not too big
      if len(diff) == 0 or diff == {''} or diff == {'n/a'} or diff == {'no requirements'} or diff == {'none'}:
        fitting_jobs.add(key) # all requirements met for the job
      elif len(diff) == 1:
        skills_missing_1[key] = diff
      elif len(diff) == 2:
        skills_missing_2[key] = diff
      elif len(diff) == 3:
        skills_missing_3[key] = diff

  with open(file) as f: # add title, company, location etc to the links for which the user meets required skills
    skills_met = dict()
    missing_1 = dict()
    missing_2 = dict()
    missing_3 = dict()
    reader = csv.reader(f)
    header = next(reader, None)  # skip table head
    for row in reader:
      if len(row) < 6:
        continue
      job_link = row[0].strip().lower()
      last_time = row[1].strip()
      title = row[5].strip()
      if len(row) > 6:
        company = row[6].strip()
      else:
        company = ""
      if len(row) > 7:
        location = row[7].strip()
      else:
        location = ""
      if job_link in fitting_jobs:
        skills_met[job_link] = [last_time,title,company,location]
      elif job_link in skills_missing_1:
        missing_1[job_link] = [last_time,title,company,location,skills_missing_1[job_link]]
      elif job_link in skills_missing_2:
        missing_2[job_link] = [last_time,title,company,location,skills_missing_2[job_link]]
      elif job_link in skills_missing_3:
        missing_3[job_link] = [last_time,title,company,location,skills_missing_3[job_link]]
  return skills_met, missing_1, missing_2, missing_3

In [127]:
skills_file = "job_skills.csv"
job_postings_file = "linkedin_job_postings.csv"
skills_dict = dict()
skills_dict, all_skills = preprocess(skills_file) # all_skills is a set of all possible skills, could add to web app. Takes ~2min to run.

0 rows processed
100000 rows processed
200000 rows processed
300000 rows processed
400000 rows processed
500000 rows processed
600000 rows processed
700000 rows processed
800000 rows processed
900000 rows processed
1000000 rows processed
1100000 rows processed
1200000 rows processed


In [130]:
input = ["example skill","microsoft","computer","money laundering"]
skills_met, missing_1, missing_2, missing_3 = get_jobs_info(job_postings_file,skills_dict, input)

In [131]:
skills_met # all jobs for which the user is qualified. An entry consists of: the LinkedIn link, date it was checked, the job title, the company and the location

{'https://www.linkedin.com/jobs/view/behavioral-health-counselor-travel-position-at-sandstone-care-3803493931': ['2024-01-21 07:40:11.491706+00',
  'Behavioral Health Counselor (Travel Position)',
  'Sandstone Care',
  'Chantilly, VA'],
 'https://www.linkedin.com/jobs/view/volunteer-bereavement-volunteer-at-cardinal-hospice-care-3803993564': ['2024-01-21 07:44:16.834727+00',
  'Volunteer: Bereavement Volunteer',
  'Cardinal Hospice Care',
  'Jacksonville, NC'],
 'https://www.linkedin.com/jobs/view/rn-per-diem-at-people-inc-3736174816': ['2024-01-21 07:13:13.433253+00',
  'RN PER DIEM',
  'People Inc.',
  'Rochester, NY'],
 'https://www.linkedin.com/jobs/view/administrative-assistant-dcn-temp-position-at-foundation-partners-group-3727249313': ['2024-01-20 13:47:25.4771+00',
  'Administrative Assistant DCN - Temp Position',
  'Foundation Partners Group',
  'Santa Ana, CA'],
 'https://www.linkedin.com/jobs/view/radiologic-technologist-f-t-day-evening-at-yale-new-haven-health-3744443972': 

In [136]:
len(skills_met)

2600